# 🧪 TPC-DS Benchmark Queries 41–45
Databricks SQL using `%run ./specify_catalog_schema`

In [0]:
%run ./specify_catalog_schema

In [0]:
%sql
-- Query 41: Identifying Products with Specific Manufacturer Attributes
-- TPC-DS Query 41
SELECT DISTINCT(i_product_name)
FROM item i1
WHERE i_manufact_id BETWEEN 742 AND 782
  AND (
    SELECT COUNT(*) AS item_cnt
    FROM item
    WHERE i_manufact = i1.i_manufact
      AND (
        (i_category = 'Women'
         AND (i_color = 'orchid' OR i_color = 'papaya')
         AND (i_units = 'Pound' OR i_units = 'Lb')
         AND (i_size = 'petite' OR i_size = 'medium'))
        OR
        (i_category = 'Women'
         AND (i_color = 'burlywood' OR i_color = 'navy')
         AND (i_units = 'Bundle' OR i_units = 'Each')
         AND (i_size = 'petite' OR i_size = 'medium'))
      )
  ) > 0
ORDER BY i_product_name
LIMIT 100;

In [0]:
%sql
-- Query 42: Sales Analysis by Category and Year
-- TPC-DS Query 42
SELECT dt.d_year,
       item.i_category_id,
       item.i_category,
       SUM(ss_ext_sales_price) AS total_sales
FROM date_dim dt,
     store_sales,
     item
WHERE dt.d_date_sk = store_sales.ss_sold_date_sk
  AND store_sales.ss_item_sk = item.i_item_sk
  AND item.i_manager_id = 1
  AND dt.d_moy = 12
  AND dt.d_year = 1998
GROUP BY dt.d_year,
         item.i_category_id,
         item.i_category
ORDER BY total_sales DESC,
         dt.d_year,
         item.i_category_id,
         item.i_category
LIMIT 100;

In [0]:
%sql
-- Query 43: Weekly Sales Performance by Store
-- TPC-DS Query 43
SELECT s_store_name,
       s_store_id,
       SUM(CASE WHEN d_day_name = 'Sunday' THEN ss_sales_price ELSE NULL END) AS sun_sales,
       SUM(CASE WHEN d_day_name = 'Monday' THEN ss_sales_price ELSE NULL END) AS mon_sales,
       SUM(CASE WHEN d_day_name = 'Tuesday' THEN ss_sales_price ELSE NULL END) AS tue_sales,
       SUM(CASE WHEN d_day_name = 'Wednesday' THEN ss_sales_price ELSE NULL END) AS wed_sales,
       SUM(CASE WHEN d_day_name = 'Thursday' THEN ss_sales_price ELSE NULL END) AS thu_sales,
       SUM(CASE WHEN d_day_name = 'Friday' THEN ss_sales_price ELSE NULL END) AS fri_sales,
       SUM(CASE WHEN d_day_name = 'Saturday' THEN ss_sales_price ELSE NULL END) AS sat_sales
FROM date_dim,
     store_sales,
     store
WHERE d_date_sk = ss_sold_date_sk
  AND s_store_sk = ss_store_sk
  AND s_gmt_offset = -5
  AND d_year = 2000
GROUP BY s_store_name,
         s_store_id
ORDER BY s_store_name,
         s_store_id,
         sun_sales,
         mon_sales,
         tue_sales,
         wed_sales,
         thu_sales,
         fri_sales,
         sat_sales
LIMIT 100;

In [0]:
%sql
-- Query 44: Promotion Effectiveness Analysis
-- TPC-DS Query 44
SELECT i_item_id,
       i_item_desc,
       p_channel_email,
       p_channel_catalog,
       p_channel_tv,
       SUM(cs_sales_price) AS catalog_sales,
       SUM(ws_sales_price) AS web_sales
FROM catalog_sales,
     promotion,
     item,
     web_sales
WHERE cs_promo_sk = p_promo_sk
  AND cs_item_sk = i_item_sk
  AND ws_item_sk = i_item_sk
GROUP BY i_item_id,
         i_item_desc,
         p_channel_email,
         p_channel_catalog,
         p_channel_tv
ORDER BY catalog_sales DESC,
         web_sales DESC
LIMIT 100;

In [0]:
%sql
-- Query 45: Sales Analysis by Customer Location
-- TPC-DS Query 45
SELECT ca_zip,
       ca_city,
       SUM(ws_sales_price) AS total_sales
FROM web_sales,
     customer,
     customer_address,
     date_dim,
     item
WHERE ws_bill_customer_sk = c_customer_sk
  AND c_current_addr_sk = ca_address_sk
  AND ws_item_sk = i_item_sk
  AND ws_sold_date_sk = d_date_sk
  AND d_qoy = 2
  AND d_year = 2000
  AND i_category = 'Jewelry'
GROUP BY ca_zip,
         ca_city
ORDER BY ca_zip,
         ca_city,
         total_sales
LIMIT 100;